In [354]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import skew
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import Lasso , Ridge, ElasticNet, BayesianRidge, LinearRegression
from sklearn.ensemble import HistGradientBoostingRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split

In [114]:
label_X_train = pd.read_csv("../Data/data for semisupervise/label_x.csv")
y_train = pd.read_csv("../Data/data for semisupervise/log_price.csv")
combine_X = pd.read_csv("../Data/data for semisupervise/combine_full.csv")

In [115]:
unlabel_X = combine_X[1460:]

In [116]:
#train test split
X_train, X_test, y_train, y_test = train_test_split(label_X_train, y_train, test_size=0.2, random_state=42)

In [117]:
#drop ID and fillna
X_train = X_train.drop(['Id'], axis = 1)
X_test = X_test.drop(['Id'], axis = 1)
unlabel_X = unlabel_X.drop(['Id'], axis = 1)
X_train = X_train.fillna(0)
X_test= X_test.fillna(0)
unlabel_X = unlabel_X.fillna(0)

In [491]:
X_train.dtype

dtype('float64')

In [300]:
y_test = y_test.to_numpy()

In [479]:
#train

regressor = RandomForestRegressor(n_estimators=1000,       # Increase number of trees
    max_depth=25,             # Increase tree depth
    min_samples_split=2,     # Decrease min samples required to split an internal node
    min_samples_leaf=2,      # Decrease min samples required to be at a leaf node
    max_features= 130    )
regressor.fit(X_train, y_train)

C:\Users\longv\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


RandomForestRegressor(max_depth=25, max_features=130, min_samples_leaf=2,
                      n_estimators=1000)

In [480]:
pred = regressor.predict(X_test)

In [481]:
pred

array([11.84350341, 12.63131778, 11.6639802 , 11.92979386, 12.65108011,
       11.3487918 , 12.25058155, 11.92262452, 11.34580198, 11.743158  ,
       11.93348168, 11.68285086, 11.62045316, 12.23875789, 12.08912246,
       11.7724216 , 12.18934788, 11.81018966, 11.66497029, 12.2482846 ,
       12.01703131, 12.28081825, 12.08625486, 11.71995323, 12.19004533,
       12.07840466, 12.13185898, 11.57734063, 12.09080646, 12.16993018,
       11.73907521, 12.46796901, 12.06868008, 11.63994475, 12.45477317,
       11.88481118, 11.85161825, 12.23315177, 12.61741779, 11.58399743,
       11.73145   , 12.36517918, 11.69656074, 12.80371423, 11.81278426,
       11.8408824 , 11.65803953, 11.76196822, 12.85130189, 11.85720198,
       11.70547759, 12.16784383, 11.69054357, 12.60091649, 11.85747593,
       12.4060299 , 12.18868646, 11.93432451, 11.87824298, 11.60910003,
       11.252788  , 11.89968317, 12.61969035, 12.4531237 , 12.56344559,
       12.25621239, 11.6147156 , 12.64179823, 11.64697611, 12.00

In [482]:
print(mean_absolute_error(pred, y_test))

0.09559483722333484


In [483]:
unlabel_pred = regressor.predict(unlabel_X)

C:\Users\longv\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:486: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(


In [484]:
unlabel_pred

array([11.73431675, 11.94023011, 12.09235221, ..., 11.95607668,
       11.66458   , 12.34525309])

In [485]:
unlabel_pred = pd.DataFrame(unlabel_pred)

In [486]:
X_combined = np.vstack([X_train, unlabel_X])
y_combined = np.concatenate([y_train, unlabel_pred])
X_combined = pd.DataFrame(X_combined)
y_combined =  pd.DataFrame(y_combined)

In [488]:
new_regressor = RandomForestRegressor(n_estimators=1500,       # Increase number of trees
    max_depth=25,             # Increase tree depth
    min_samples_split=2,     # Decrease min samples required to split an internal node
    min_samples_leaf=2,      # Decrease min samples required to be at a leaf node
    max_features= 130    )

In [489]:
new_regressor.fit(X_combined, y_combined)

C:\Users\longv\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


KeyboardInterrupt: 

In [399]:
pred_final = new_regressor.predict(X_test)

In [400]:
pred_final

array([11.83557549, 12.63755705, 11.68066118, 11.91651885, 12.6296494 ,
       11.32488369, 12.21527707, 11.91848878, 11.32783667, 11.73784011,
       11.88375963, 11.70719259, 11.6457565 , 12.25794189, 12.10849949,
       11.77046403, 12.1808255 , 11.81034301, 11.65007663, 12.24314272,
       12.04658584, 12.28031748, 12.09081054, 11.71376587, 12.1927954 ,
       12.08401108, 12.1244004 , 11.59697716, 12.09639753, 12.19759095,
       11.74891721, 12.43911545, 12.06126539, 11.65151435, 12.45534239,
       11.86491861, 11.85868601, 12.24590858, 12.60791996, 11.60474441,
       11.72953324, 12.33676386, 11.7114318 , 12.80171157, 11.79888821,
       11.84519465, 11.67958084, 11.75965417, 12.87142781, 11.86392637,
       11.70706271, 12.1499127 , 11.68803964, 12.60432126, 11.85943154,
       12.38483002, 12.18913425, 11.93384795, 11.88085828, 11.60858132,
       11.24095344, 11.88973631, 12.64521384, 12.43936733, 12.58337865,
       12.24298877, 11.62526681, 12.62639298, 11.66381445, 12.02

In [401]:
print(mean_absolute_error(pred_final, y_test))

0.09852136304256771


In [402]:
print(mean_absolute_error(new_regressor.predict(X_combined), y_combined))

0.018016529755194566
